In [1]:
import nltk
from nltk.stem.porter import *
from torch.nn import *
from torch.optim import *
import numpy as np
import pandas as pd
import torch,torchvision
import random
from tqdm import *
from torch.utils.data import Dataset,DataLoader
stemmer = PorterStemmer()

In [2]:
data = pd.read_csv('./train_data_cleaning.csv')

In [3]:
def tokenize(sentence):
    return nltk.word_tokenize(sentence)

In [4]:
tokenize('test what why how')

['test', 'what', 'why', 'how']

In [5]:
def stem(word):
    return stemmer.stem(word.lower())

In [6]:
stem('test')

'test'

In [7]:
def bag_of_words(tokenized_word,all_words):
    tokenized_word = [stem(w) for w in tokenized_word]
    bag = np.zeros(len(all_words),dtype=np.float32)
    for idx,w in enumerate(all_words):
        if w in tokenized_word:
            bag[idx] = 1.0
    return bag

In [8]:
bag_of_words(['test','what !!!'],['test','what','what !!!'])

array([1., 0., 1.], dtype=float32)

In [9]:
X = data['text'].tolist()

In [10]:
y = data['target'].tolist()

In [11]:
all_words = []
new_data = []

In [12]:
tags = {}

In [13]:
for X_batch,y_batch in tqdm(zip(X,y)):
    X_batch = tokenize(X_batch)
    new_X = []
    for X_batch_in_X_batch in X_batch:
        new_X.append(stem(X_batch_in_X_batch))
    all_words.extend(new_X)
    new_data.append((new_X,y_batch))
    if y_batch not in list(tags.keys()):
        tags[y_batch] = 1
    else:
        tags[y_batch] += 1

7613it [00:02, 3158.06it/s]


In [14]:
tags

{1: 3271, 0: 4342}

In [15]:
tags = [0,1]

In [16]:
np.random.shuffle(all_words)
np.random.shuffle(new_data)

In [17]:
len(all_words)

132212

In [18]:
all_words = sorted(set(all_words))

In [19]:
len(all_words)

13928

In [20]:
X = []
y = []

In [21]:
for sentence,tag in tqdm(new_data):
    bag = bag_of_words(sentence,all_words)
    X.append(bag)
    label = tags.index(tag)
    y.append(label)

100%|██████████| 7613/7613 [00:22<00:00, 331.36it/s]


In [22]:
from sklearn.model_selection import *

In [23]:
X_train,X_test,y_train,y_test = train_test_split(X,y,test_size=0.25)

In [24]:
device = 'cuda'

In [25]:
X_train = torch.from_numpy(np.array(X_train)).to(device).float()
X_test = torch.from_numpy(np.array(X_test)).to(device).float()
y_train = torch.from_numpy(np.array(y_train)).to(device).float()
y_test = torch.from_numpy(np.array(y_test)).to(device).float()

In [26]:
def get_loss(model,X,y,criterion):
    preds = model(X)
    loss = criterion(preds.view(-1,1),y.view(-1,1))
    return loss.item()

In [27]:
def get_accuracy(model,X,y):
    preds = model(X)
    correct = 0
    total = 0
    for pred,y_batch in zip(preds,y):
        pred = int(torch.round(pred))
        y_batch = int(y_batch)
        if pred == y_batch:
            correct += 1
        total += 1
    acc = round(correct / total,3)*100
    return acc

In [28]:
class Model(Module):
    def __init__(self,hidden=512+256):
        super().__init__()
        self.activation = ReLU()
        self.linear1 = Linear(len(all_words),hidden)
        self.linear2 = Linear(hidden,hidden*2)
        self.linear3 = Linear(hidden*2,hidden*4)
        self.linear4 = Linear(hidden*4,hidden*8)
        self.linear5 = Linear(hidden*8,1)
        self.output = Linear(1,1)
        self.output_ac = Sigmoid()
    
    def forward(self,X):
        preds = self.activation(self.linear1(X))
        preds = self.activation(self.linear2(preds))
        preds = self.activation(self.linear3(preds))
        preds = self.activation(self.linear4(preds))
        preds = self.activation(self.linear5(preds))
        preds = self.output_ac(self.output(preds))
        return preds

In [29]:
model = Model().to(device)

In [30]:
criterion = BCELoss()

In [31]:
optimizer = Adam(model.parameters(),lr=0.001)

In [32]:
epochs = 1000

In [33]:
batch_size = 32

In [34]:
import wandb

In [ ]:
wandb.init(project='NLP-with-Disaster-Tweets-Clf-V2',name='baseline')
for epoch in tqdm(range(epochs)):
    for idx in range(0,len(X_train),batch_size):
        X_batch = X_train[idx:idx+batch_size].to(device).float()
        y_batch = y_train[idx:idx+batch_size].to(device).float()
        preds = model(X_batch)
        preds = preds.to(device)
        loss = criterion(preds.view(-1,1),y_batch.view(-1,1))
        optimizer.zero_grad()
        loss.backward()
        optimizer.step()
    wandb.log({'Loss':get_loss(model,X_train,y_train,criterion)})
    wandb.log({'Val Loss':get_loss(model,X_test,y_test,criterion)})
    wandb.log({'Acc':get_accuracy(model,X_train,y_train)})
    wandb.log({'Val Acc':get_accuracy(model,X_test,y_test)})
wandb.finish()

wandb: Currently logged in as: ranuga-d (use `wandb login --relogin` to force relogin)
wandb: wandb version 0.12.1 is available!  To upgrade, please run:
wandb:  $ pip install wandb --upgrade


 16%|█▌        | 162/1000 [07:19<38:11,  2.74s/it]